In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torch
from torch import nn
from importlib.util import find_spec
if find_spec("text_recognizer") is None:
    import sys
    sys.path.append('..')
    

In [2]:
import attr

In [3]:
from hydra import compose, initialize
from omegaconf import OmegaConf
from hydra.utils import instantiate

In [4]:
# context initialization
with initialize(config_path="../training/conf/model/mapping", job_name="test_app"):
    cfg = compose(config_name="word_piece")
    print(OmegaConf.to_yaml(cfg))
    print(cfg)

_target_: text_recognizer.data.mappings.WordPieceMapping
num_features: 1000
tokens: iamdb_1kwp_tokens_1000.txt
lexicon: iamdb_1kwp_lex_1000.txt
data_dir: null
use_words: false
prepend_wordsep: false
special_tokens:
- <s>
- <e>
- <p>
extra_symbols:
- '

  '

{'_target_': 'text_recognizer.data.mappings.WordPieceMapping', 'num_features': 1000, 'tokens': 'iamdb_1kwp_tokens_1000.txt', 'lexicon': 'iamdb_1kwp_lex_1000.txt', 'data_dir': None, 'use_words': False, 'prepend_wordsep': False, 'special_tokens': ['<s>', '<e>', '<p>'], 'extra_symbols': ['\n']}


In [5]:
tt =instantiate(cfg)

2021-07-30 23:08:27.495 | DEBUG    | text_recognizer.data.mappings:__attrs_post_init__:89 - Using data dir: /home/aktersnurra/projects/text-recognizer/data/downloaded/iam/iamdb


In [6]:
tt

WordPieceMapping(extra_symbols={'\n'}, mapping=['<b>', '<s>', '<e>', '<p>', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '!', '"', '#', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '?', '\n'], inverse_mapping={'<b>': 0, '<s>': 1, '<e>': 2, '<p>': 3, '0': 4, '1': 5, '2': 6, '3': 7, '4': 8, '5': 9, '6': 10, '7': 11, '8': 12, '9': 13, 'A': 14, 'B': 15, 'C': 16, 'D': 17, 'E': 18, 'F': 19, 'G': 20, 'H': 21, 'I': 22, 'J': 23, 'K': 24, 'L': 25, 'M': 26, 'N': 27, 'O': 28, 'P': 29, 'Q': 30, 'R': 31, 'S': 32, 'T': 33, 'U': 34, 'V': 35, 'W': 36, 'X': 37, 'Y': 38, 'Z': 39, 'a': 40, 'b': 41, 'c': 42, 'd': 43, 'e': 44, 'f': 45, 'g': 46, 'h': 47, 'i': 48, 'j': 49, 'k': 50, 'l': 51, 'm': 52, 'n': 53, 'o': 54, 'p': 55, 'q'

In [ ]:
tt.mapping

In [ ]:
# context initialization
with initialize(config_path="../training/conf/model/", job_name="test_app"):
    cfg = compose(config_name="lit_transformer")
    print(OmegaConf.to_yaml(cfg))
    print(cfg)

In [ ]:
# context initialization
with initialize(config_path="../training/conf/network/", job_name="test_app"):
    cfg = compose(config_name="conv_transformer")
    print(OmegaConf.to_yaml(cfg))
    print(cfg)

In [ ]:
from importlib import import_module

In [ ]:
x = import_module("text_recognizer.networks.transformer.attention")

In [ ]:
y = partial(getattr(x, "Attention"), dim=16, num_heads=2, **cfg.decoder.attn_kwargs)

In [ ]:
y().causal

In [ ]:
y(causal=True)

In [ ]:
net = instantiate(cfg)

In [ ]:
net